In [1]:
from pathlib import Path


import evaluation_utils as eval_u
import deeplab_utils as dl_u
import OCR_utils as OCR_u
import image_utils as image_u
import mapping_utils as mapping_u

import difflib
import os

import shapely
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.geometry import mapping, shape

import matplotlib.pyplot as plt

from mxnet import image

import json
import string
import numpy as np

import time

Using TensorFlow backend.
[nltk_data] Downloading package maxent_ne_chunker to C:\Users\Agathe
[nltk_data]     Balayn\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to C:\Users\Agathe
[nltk_data]     Balayn\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
#inputs_to_mapping_OCR

In [ ]:
#input_image_file = '2017_11534654.jpg'

# Hyper-parameters of our method
mapping_type = "simple_rule"
methods_to_account = ["semantic_segmentation", "OCR"]
type_semantic_seg_rule = "simple_list"
type_OCR_rule = "simple_rule"
type_scene_rule = "todo"
save_groundtruth = True
prepare_tools = True
ground_truth_file = '../val2017.json'

# Test with 10 test images.
path_image_folder = '../dataset_images/'
pathlist = Path(path_image_folder).glob('**/*.*')


# Prepare recurring tools.
if prepare_tools:
    dict_preparation = mapping_u.prepare_needed_elements(methods_to_account)
    needed_input = mapping_u.prepare_OCR_name_matching()

nb_image = 0
# Process each image.

#pathlist = ['2017_11534654.jpg']
for input_image_file in pathlist:
    start = time.time()
    nb_image += 1
    print("Processing image nb", str(nb_image), input_image_file)
    if nb_image >= 65:
        # Process the images to get potential private elements
        inputs_to_mapping_semantic_segmentation, inputs_to_mapping_OCR, inputs_to_mapping_scene = \
        mapping_u.get_input_to_mapping(input_image_file, methods_to_account, dict_preparation)
        print(inputs_to_mapping_OCR)

        # Filter the elements with the mapping
        if len(inputs_to_mapping_semantic_segmentation) == 0:
            inputs_to_mapping_semantic_segmentation = [[]]
        #print(inputs_to_mapping_OCR)
        if len(inputs_to_mapping_OCR) == 0:
            inputs_to_mapping_OCR = [[]]
        #print(inputs_to_mapping_OCR)
        polys_to_obfuscate = mapping_u.ruleBasedMapping(type_semantic_seg_rule, type_OCR_rule, type_scene_rule, inputs_to_mapping_semantic_segmentation[0], inputs_to_mapping_OCR[0], inputs_to_mapping_scene, needed_input)

        # Post-process the final polygons
        if len(polys_to_obfuscate) > 1:
            polys_to_obfuscate_final = inage_u.post_process_polygons(polys_to_obfuscate)
        else:
            polys_to_obfuscate_final = polys_to_obfuscate

        # Save the polygons
        image_u.save_polygons("simple_mapping_poly_09.json", polys_to_obfuscate_final, Path(input_image_file).stem)

        # Save the images
        # Prediction polygons
        save_file = Path("simple_mapping_images_09/") / (Path(input_image_file).stem + ".png")
        #printPolygons(input_image_file, polys_to_obfuscate, save_file)
        image_u.printPolygons(input_image_file, polys_to_obfuscate_final, save_file)

        # Ground truth polygons
        if save_groundtruth:
            list_GT_polygons = image_u.prepare_GT_polygon(input_image_file, ground_truth_file)
            save_file = Path("ground_truth_images/") / (Path(input_image_file).stem + ".png")
            #printPolygons(input_image_file, polys_to_obfuscate, save_file)
            image_u.printPolygons(input_image_file, list_GT_polygons, save_file)

    end = time.time()
    print("Time to process image: ", end - start)

# Evaluation

In [3]:
# Read the ground truth file.
with open(Path('../val2017.json'), 'r') as f:
    ground_truth = json.load(f)
ground_truth = ground_truth['annotations']
# Read the prediction file.
with open(Path("simple_mapping_poly_09.json"), 'r') as f:
    list_predictions = json.load(f)
list_predictions = list_predictions["list_image_poly"]


In [4]:
prediction_list = list_predictions
ground_truth_list = ground_truth

In [5]:
rate_obuscation_per_privacy_element = eval_u.evaluationPerPrivacyElement(prediction_list, ground_truth_list)

In [6]:
#print(rate_obuscation_per_privacy_element)
print(eval_u.aggregateResultsPrivacyElement(rate_obuscation_per_privacy_element))

{'a111_name_all': [0.0,
  0.0,
  0.0,
  0.0,
  0.6946169278102453,
  0.022445561139028476,
  0.9396907216494845,
  1.0,
  1.0,
  0.4403234985048201,
  1.0,
  0.5633606747497992,
  0.9874247389680665,
  0.20376560002229138,
  0.20289738198784596,
  1.0,
  0.1382046568627451,
  0.4560247167868177,
  0.43411107459743675,
  0.36220328667072427,
  0.6570796225614928,
  0.4690899202320522,
  0.0,
  0.9413145539906103,
  0.8197658063069567,
  0.9898415278342138,
  0.0,
  0.47455902306648573,
  0.6235294117647059,
  1.0,
  0.9820971867007673,
  1.0,
  0.995180258338153,
  0.34068273538450144,
  1.0,
  0.13576320939334638,
  0.6016890213611525,
  0.3586110488363819,
  0.15758295359918859,
  0.1836734693877551,
  0.75,
  0.12670190613487106,
  0.31699470098410293,
  0.15376782077393075,
  0.2432928172562091,
  0.7282608695652174,
  0.5867143086029367,
  0.9743017985085539,
  0.5208333333333334,
  0.44155844155844154,
  0.8235177532288183,
  0.46556900726392253,
  0.9598522754348344,
  0.90958412

In [ ]:

prediction_list = list_predictions
ground_truth_list = ground_truth

segment_list= [2, 4, 8, 12, 16, 20]

dict_results = eval_u.evaluationPerPixel(prediction_list, ground_truth_list, segment_list)

# Aggregate the results
dict_results_agg = eval_u.aggregateResultsPrivacyElement(dict_results)
dict_results_agg_no_nan = eval_u.aggregateResultsPrivacyElement(dict_results, "nonan")

In [ ]:
 print(dict_results_agg)
print(dict_results_agg_no_nan)

# Mining_rules

In [2]:
# Get random data for training.
image_folder = "../train2017"
nb_files = 200
import os
import random

def getkRandomImages(nb_files, image_folder):
    files = os.listdir(image_folder)
    index = random.sample(range(0, len(files)), nb_files)
    return [files[i] for i in index]
list_images = getkRandomImages(nb_files, image_folder)

In [4]:
from functools import singledispatch


@singledispatch
def to_serializable(val):
    """Used by default."""
    return str(val)

@to_serializable.register(np.float32)
def ts_float32(val):
    """Used if *val* is an instance of numpy.float32."""
    return np.float64(val)

def addElementToJson(element, json_file):
    if not os.path.isfile(json_file):
        with open(json_file, 'w') as fp:
            json.dump({"list_json": []}, fp)
    with open(json_file) as j_file: 
        data = json.load(j_file) 
        temp = data['list_json'] 
        temp.append({element[0]: element[1]}) 
    with open(json_file,'w') as f: 
        json.dump(data, f, indent=4, default=to_serializable)    

def getSceneJson(image_folder, list_image_files, output_json, dict_param):
    nb_im = 0
    for im in list_image_files:
        nb_im +=1
        print("Image no. ", str(nb_im), ": ", im)
        _, _, inputs_to_mapping_scene = \
                mapping_u.get_input_to_mapping(Path(image_folder + "/" + im), ["scene_recognition"], dict_param)
        addElementToJson((im, inputs_to_mapping_scene[0]), output_json)

output_json = "200_training_scene_info.json"
getSceneJson(image_folder, list_images, output_json, {"top_k":-1})

Image no.  1 :  2017_96543952.jpg
(3456, 4608, 3)
Image no.  2 :  2017_53009285.jpg
(1200, 1600, 3)
Image no.  3 :  2017_19603914.jpg
(5616, 3744, 3)
Image no.  4 :  2017_29493933.jpg
(2592, 1944, 3)
Image no.  5 :  2017_96345674.jpg
(480, 640, 3)
Image no.  6 :  2017_60051448.jpg
(428, 640, 3)
Image no.  7 :  2017_38342719.jpg
(2048, 1365, 3)
Image no.  8 :  2017_42397883.jpg
(2252, 3124, 3)
Image no.  9 :  2017_17736107.jpg
(284, 549, 3)
Image no.  10 :  2017_73844969.jpg
(5906, 4724, 3)
Image no.  11 :  2017_16092595.jpg
(2948, 4242, 3)
Image no.  12 :  2017_48671800.jpg
(1730, 4352, 3)
Image no.  13 :  2017_87768422.png
(384, 794, 3)
Image no.  14 :  2017_84922543.jpg
(505, 1294, 3)
Image no.  15 :  2017_69146435.jpg
(2160, 3248, 3)
Image no.  16 :  2017_78837389.jpg
(1080, 1080, 3)
Image no.  17 :  2017_97503841.jpg
(742, 1496, 3)
Image no.  18 :  2017_77600966.jpg
(5184, 3456, 3)
Image no.  19 :  2017_55093320.jpg
(768, 1024, 3)
Image no.  20 :  2017_62093290.jpg
(720, 1280, 3)
I

Image no.  163 :  2017_41517546.jpg
(2448, 3264, 3)
Image no.  164 :  2017_70926607.jpg
(3456, 4608, 3)
Image no.  165 :  2017_33011529.jpg
(600, 800, 3)
Image no.  166 :  2017_59303968.jpg
(640, 640, 3)
Image no.  167 :  2017_64007922.jpg
(2030, 3051, 3)
Image no.  168 :  2017_63578552.jpg
(1128, 1704)
Image no.  169 :  2017_20806393.jpg
(2400, 1800, 3)
Image no.  170 :  2017_11741448.jpg
(2048, 1536, 3)
Image no.  171 :  2017_75499087.jpg
(1568, 2784, 3)
Image no.  172 :  2017_41221635.jpg
(4000, 3000, 3)
Image no.  173 :  2017_59260971.jpg
(2448, 3264, 3)
Image no.  174 :  2017_24440996.jpg
(3456, 5184, 3)
Image no.  175 :  2017_38694483.jpg
(768, 1024, 3)
Image no.  176 :  2017_70578199.jpg
(1500, 2100, 3)
Image no.  177 :  2017_21682050.jpg
(629, 400, 3)
Image no.  178 :  2017_93365945.jpg
(480, 641, 3)
Image no.  179 :  2017_24422940.jpg
(3056, 4576, 3)
Image no.  180 :  2017_37846449.jpg
(960, 1280, 3)
Image no.  181 :  2017_77772829.jpg
(2297, 3445, 3)
Image no.  182 :  2017_73

In [12]:
def getGroundTruthJson(image_folder, list_image_files, output_json, ground_truth_file):
    for im in list_image_files:
        addElementToJson((im, list(set(getGroundTruthLabel(im, ground_truth_file)))), output_json)
getGroundTruthJson(image_folder, list_images, "GT_200_training_images.json", '../train2017.json')        

In [11]:


# Get ground truth labels.
def getGroundTruthLabel(image_file, ground_truth_file):
    with open(Path(ground_truth_file), 'r') as f:
        ground_truth = json.load(f)
    ground_truth = ground_truth['annotations']
    image_ground_truth = ground_truth[Path(image_file).stem]["attributes"]
    # Make a list of the GT elements.
    list_private_info = []
    for element in image_ground_truth:
        list_private_info.append(element["attr_id"])
    return list_private_info

# Create input list for our own dataset

def createMiningInputs(type_of_inputs, ground_truth_file, path_image_folder, dict_param = {"top_k":1}, nb_image_start=0):
   
    pathlist = Path(path_image_folder).glob('**/*.*')

    nb_image = 0

    list_semantic = []
    list_scene = []
    list_GT_infos = []

    for input_image_file in pathlist:
        start = time.time()
        nb_image += 1
        print("Processing image nb", str(nb_image), input_image_file)
        if nb_image >= nb_image_start:
            # Process the images to get potential private elements
            if "semantic_segmentation" in type_of_inputs:
                inputs_to_mapping_semantic_segmentation, _, _ = \
                mapping_u.get_input_to_mapping(input_image_file, "semantic_segmentation", _)
                # GEt only the classes names, and unique ones (in case there are repetitions of the same class.)
                post_processed_list = []
                for element in inputs_to_mapping_semantic_segmentation[0]:
                    post_processed_list.append(element[1])
                list_semantic.append(list(set(post_processed_list)))
                
            if "scene_recognition" in type_of_inputs:
                _, _, inputs_to_mapping_scene = \
                mapping_u.get_input_to_mapping(input_image_file, "scene_recognition", dict_param)
                list_scene.append(inputs_to_mapping_scene[0][0][0])
        
            if "ground_truth" in type_of_inputs:
                list_GT_infos.append(list(set(getGroundTruthLabel(input_image_file, ground_truth_file))))
                                          
            if "OCR" in type_of_inputs:
                print("TODO: OCR")

In [12]:

# One list of the predictions by the ML model (later also OCR and scene segmentation?) 


## semantic segmentaiton
# Hyper-parameters of our method
methods_to_account = ["semantic_segmentation"]

ground_truth_file = '../val2017.json'


path_image_folder = '../dataset_images/'
pathlist = Path(path_image_folder).glob('**/*.*')

nb_image = 0

list_semantic = []


for input_image_file in pathlist:
    start = time.time()
    nb_image += 1
    print("Processing image nb", str(nb_image), input_image_file)
    if nb_image >= 0:
        # Process the images to get potential private elements
        inputs_to_mapping_semantic_segmentation, _, _ = \
        mapping_u.get_input_to_mapping(input_image_file, methods_to_account, _)
        # GEt only the classes names, and unique ones (in case there are repetitions of the same class.)
        post_processed_list = []
        for element in inputs_to_mapping_semantic_segmentation[0]:
            post_processed_list.append(element[1])
        list_semantic.append(list(set(post_processed_list)))
        
### scene recognition

# and one list with the ground truth elements 

Processing image nb 1 ..\dataset_images\2017_10024630.jpg
TODO: choose whether to get the probability
predicting
calculating labels
calculating probabilities.
Creating masks.
Resizing image prediction.
Number of classes with masks to create:  3
Processing image nb 2 ..\dataset_images\2017_10090671.jpg
TODO: choose whether to get the probability
predicting
calculating labels
calculating probabilities.
Creating masks.
Resizing image prediction.
Number of classes with masks to create:  8
Processing image nb 3 ..\dataset_images\2017_10099229.jpg
TODO: choose whether to get the probability
predicting
calculating labels
calculating probabilities.
Creating masks.
Resizing image prediction.
Number of classes with masks to create:  7
Processing image nb 4 ..\dataset_images\2017_10297642.jpg
TODO: choose whether to get the probability
predicting
calculating labels
calculating probabilities.
Creating masks.
Resizing image prediction.
Number of classes with masks to create:  10
Processing image nb

predicting
calculating labels
calculating probabilities.
Creating masks.
Resizing image prediction.
Number of classes with masks to create:  10
Processing image nb 35 ..\dataset_images\2017_16133375.jpg
TODO: choose whether to get the probability
predicting
calculating labels
calculating probabilities.
Creating masks.
Resizing image prediction.
Number of classes with masks to create:  8
Processing image nb 36 ..\dataset_images\2017_16724573.jpg
TODO: choose whether to get the probability
predicting
calculating labels
calculating probabilities.
Creating masks.
Resizing image prediction.
Number of classes with masks to create:  16
Processing image nb 37 ..\dataset_images\2017_16766682.jpg
TODO: choose whether to get the probability
predicting
calculating labels
calculating probabilities.
Creating masks.
Resizing image prediction.
Number of classes with masks to create:  5
Processing image nb 38 ..\dataset_images\2017_17731795.jpg
TODO: choose whether to get the probability
predicting
cal

predicting
calculating labels
calculating probabilities.
Creating masks.
Resizing image prediction.
Number of classes with masks to create:  8
Processing image nb 69 ..\dataset_images\2017_29789049.jpg
TODO: choose whether to get the probability
predicting
calculating labels
calculating probabilities.
Creating masks.
Resizing image prediction.
Number of classes with masks to create:  11
Processing image nb 70 ..\dataset_images\2017_29854712.jpg
TODO: choose whether to get the probability
predicting
calculating labels
calculating probabilities.
Creating masks.
Resizing image prediction.
Number of classes with masks to create:  7
Processing image nb 71 ..\dataset_images\2017_31542591.jpg
TODO: choose whether to get the probability
predicting
calculating labels
calculating probabilities.
Creating masks.
Resizing image prediction.
Number of classes with masks to create:  5
Processing image nb 72 ..\dataset_images\2017_31564686.jpg
TODO: choose whether to get the probability
predicting
calc

predicting
calculating labels
calculating probabilities.
Creating masks.
Resizing image prediction.
Number of classes with masks to create:  10
Processing image nb 103 ..\dataset_images\2017_46428064.jpg
TODO: choose whether to get the probability
predicting
calculating labels
calculating probabilities.
Creating masks.
Resizing image prediction.
Number of classes with masks to create:  4
Processing image nb 104 ..\dataset_images\2017_46752720.jpg
TODO: choose whether to get the probability
predicting
calculating labels
calculating probabilities.
Creating masks.
Resizing image prediction.
Number of classes with masks to create:  9
Processing image nb 105 ..\dataset_images\2017_47011563.jpg
TODO: choose whether to get the probability
predicting
calculating labels
calculating probabilities.
Creating masks.
Resizing image prediction.
Number of classes with masks to create:  2
Processing image nb 106 ..\dataset_images\2017_47355548.jpg
TODO: choose whether to get the probability
predicting


Creating masks.
Resizing image prediction.
Number of classes with masks to create:  5
Processing image nb 136 ..\dataset_images\2017_62255732.jpg
TODO: choose whether to get the probability
predicting
calculating labels
calculating probabilities.
Creating masks.
Resizing image prediction.
Number of classes with masks to create:  10
Processing image nb 137 ..\dataset_images\2017_62795821.jpg
TODO: choose whether to get the probability
predicting
calculating labels
calculating probabilities.
Creating masks.
Resizing image prediction.
Number of classes with masks to create:  8
Processing image nb 138 ..\dataset_images\2017_63114311.jpg
TODO: choose whether to get the probability
predicting
calculating labels
calculating probabilities.
Creating masks.
Resizing image prediction.
Number of classes with masks to create:  5
Processing image nb 139 ..\dataset_images\2017_63543770.jpg
TODO: choose whether to get the probability
predicting
calculating labels
calculating probabilities.
Creating ma

Number of classes with masks to create:  12
Processing image nb 169 ..\dataset_images\2017_81977834.jpg
TODO: choose whether to get the probability
predicting
calculating labels
calculating probabilities.
Creating masks.
Resizing image prediction.
Number of classes with masks to create:  8
Processing image nb 170 ..\dataset_images\2017_84153080.jpg
TODO: choose whether to get the probability
predicting
calculating labels
calculating probabilities.
Creating masks.
Resizing image prediction.
Number of classes with masks to create:  22
Processing image nb 171 ..\dataset_images\2017_84596196.jpg
TODO: choose whether to get the probability
predicting
calculating labels
calculating probabilities.
Creating masks.
Resizing image prediction.
Number of classes with masks to create:  14
Processing image nb 172 ..\dataset_images\2017_84835300.jpg
TODO: choose whether to get the probability
predicting
calculating labels
calculating probabilities.
Creating masks.
Resizing image prediction.
Number of

Processing image nb 202 ..\dataset_images\2017_98018612.jpg
TODO: choose whether to get the probability
predicting
calculating labels
calculating probabilities.
Creating masks.
Resizing image prediction.
Number of classes with masks to create:  4
Processing image nb 203 ..\dataset_images\2017_98143522.jpg
TODO: choose whether to get the probability
predicting
calculating labels
calculating probabilities.
Creating masks.
Resizing image prediction.
Number of classes with masks to create:  8
Processing image nb 204 ..\dataset_images\2017_98561452.jpg
TODO: choose whether to get the probability
predicting
calculating labels
calculating probabilities.
Creating masks.
Resizing image prediction.
Number of classes with masks to create:  14
Processing image nb 205 ..\dataset_images\2017_98778527.jpg
TODO: choose whether to get the probability
predicting
calculating labels
calculating probabilities.
Creating masks.
Resizing image prediction.
Number of classes with masks to create:  16
Processing

In [2]:
methods_to_account = ["scene_recognition"]

ground_truth_file = '../val2017.json'

# Test with 10 test images.
path_image_folder = '../dataset_images/'
pathlist = Path(path_image_folder).glob('**/*.*')
dict_param = {"top_k":1}

nb_image = 0
# Process each image.
list_scene = []
for input_image_file in pathlist:
    start = time.time()
    nb_image += 1
    print("Processing image nb", str(nb_image), input_image_file)
    if nb_image >= 0 :
        # Process the images to get potential private elements
        _, _, inputs_to_mapping_scene = \
        mapping_u.get_input_to_mapping(input_image_file, methods_to_account, dict_param)
        list_scene.append(inputs_to_mapping_scene[0][0][0])
        


Processing image nb 1 ..\dataset_images\2017_10024630.jpg
Processing image nb 2 ..\dataset_images\2017_10090671.jpg
Processing image nb 3 ..\dataset_images\2017_10099229.jpg
Processing image nb 4 ..\dataset_images\2017_10297642.jpg
Processing image nb 5 ..\dataset_images\2017_10356759.jpg
Processing image nb 6 ..\dataset_images\2017_10649357.jpg
Processing image nb 7 ..\dataset_images\2017_10841739.jpg
Processing image nb 8 ..\dataset_images\2017_10857303.jpg
Processing image nb 9 ..\dataset_images\2017_10904878.jpg
Processing image nb 10 ..\dataset_images\2017_10950067.jpg
Processing image nb 11 ..\dataset_images\2017_10992021.jpg
Processing image nb 12 ..\dataset_images\2017_11231095.jpg
Processing image nb 13 ..\dataset_images\2017_11301779.jpg
Processing image nb 14 ..\dataset_images\2017_11470792.jpg
Processing image nb 15 ..\dataset_images\2017_11700602.jpg
Processing image nb 16 ..\dataset_images\2017_11807522.jpg
Processing image nb 17 ..\dataset_images\2017_12120917.png
Proces

Processing image nb 140 ..\dataset_images\2017_63599730.jpg
Processing image nb 141 ..\dataset_images\2017_65346554.jpg
Processing image nb 142 ..\dataset_images\2017_65952416.jpg
Processing image nb 143 ..\dataset_images\2017_66405707.jpg
Processing image nb 144 ..\dataset_images\2017_66745676.jpg
Processing image nb 145 ..\dataset_images\2017_67076916.jpg
Processing image nb 146 ..\dataset_images\2017_68712801.jpg
Processing image nb 147 ..\dataset_images\2017_68942629.jpg
Processing image nb 148 ..\dataset_images\2017_69238495.jpg
Processing image nb 149 ..\dataset_images\2017_70153172.jpg
Processing image nb 150 ..\dataset_images\2017_72070545.jpg
Processing image nb 151 ..\dataset_images\2017_72101053.jpg
Processing image nb 152 ..\dataset_images\2017_72388401.jpg
Processing image nb 153 ..\dataset_images\2017_72621662.jpg
Processing image nb 154 ..\dataset_images\2017_73074722.jpg
Processing image nb 155 ..\dataset_images\2017_74028401.jpg
Processing image nb 156 ..\dataset_image

In [8]:
# And one list with the ground truth elements. 


# Get ground truth labels.
def getGroundTruthLabel(image_file, ground_truth_file):
    with open(Path(ground_truth_file), 'r') as f:
        ground_truth = json.load(f)
    ground_truth = ground_truth['annotations']
    image_ground_truth = ground_truth[Path(input_image_file).stem]["attributes"]
    # Make a list of the GT elements.
    list_private_info = []
    for element in image_ground_truth:
        list_private_info.append(element["attr_id"])
    return list_private_info

ground_truth_file = '../val2017.json'

# Test with 10 test images.
path_image_folder = '../dataset_images/'
pathlist = Path(path_image_folder).glob('**/*.*')

nb_image = 0
# Process each image.
list_GT_infos = []
for input_image_file in pathlist:
    start = time.time()
    nb_image += 1
    print("Processing image nb", str(nb_image), input_image_file)
    if nb_image >= 0:
        list_GT_infos.append(list(set(getGroundTruthLabel(input_image_file, ground_truth_file))))

Processing image nb 1 ..\dataset_images\2017_10024630.jpg
Processing image nb 2 ..\dataset_images\2017_10090671.jpg
Processing image nb 3 ..\dataset_images\2017_10099229.jpg
Processing image nb 4 ..\dataset_images\2017_10297642.jpg
Processing image nb 5 ..\dataset_images\2017_10356759.jpg
Processing image nb 6 ..\dataset_images\2017_10649357.jpg
Processing image nb 7 ..\dataset_images\2017_10841739.jpg
Processing image nb 8 ..\dataset_images\2017_10857303.jpg
Processing image nb 9 ..\dataset_images\2017_10904878.jpg
Processing image nb 10 ..\dataset_images\2017_10950067.jpg
Processing image nb 11 ..\dataset_images\2017_10992021.jpg
Processing image nb 12 ..\dataset_images\2017_11231095.jpg
Processing image nb 13 ..\dataset_images\2017_11301779.jpg
Processing image nb 14 ..\dataset_images\2017_11470792.jpg
Processing image nb 15 ..\dataset_images\2017_11700602.jpg
Processing image nb 16 ..\dataset_images\2017_11807522.jpg
Processing image nb 17 ..\dataset_images\2017_12120917.png
Proces

Processing image nb 140 ..\dataset_images\2017_63599730.jpg
Processing image nb 141 ..\dataset_images\2017_65346554.jpg
Processing image nb 142 ..\dataset_images\2017_65952416.jpg
Processing image nb 143 ..\dataset_images\2017_66405707.jpg
Processing image nb 144 ..\dataset_images\2017_66745676.jpg
Processing image nb 145 ..\dataset_images\2017_67076916.jpg
Processing image nb 146 ..\dataset_images\2017_68712801.jpg
Processing image nb 147 ..\dataset_images\2017_68942629.jpg
Processing image nb 148 ..\dataset_images\2017_69238495.jpg
Processing image nb 149 ..\dataset_images\2017_70153172.jpg
Processing image nb 150 ..\dataset_images\2017_72070545.jpg
Processing image nb 151 ..\dataset_images\2017_72101053.jpg
Processing image nb 152 ..\dataset_images\2017_72388401.jpg
Processing image nb 153 ..\dataset_images\2017_72621662.jpg
Processing image nb 154 ..\dataset_images\2017_73074722.jpg
Processing image nb 155 ..\dataset_images\2017_74028401.jpg
Processing image nb 156 ..\dataset_image

In [15]:
# Rule mining

In [227]:
### Create dataset

# dataset of semantic and ground truth
dataset_semantic = []
for elem_semantic, elem_GT in zip(list_semantic, list_GT_infos):
    dataset_semantic.append(elem_semantic + elem_GT)

# dataset of semantic, scene, ground truth
dataset_semantic_scene = []
for elem_semantic, elem_scene, elem_GT in zip(list_semantic, list_scene, list_GT_infos):
    dataset_semantic_scene.append(elem_semantic + [elem_scene[0]] + elem_GT)

# dataset of semantic, OCR, scene, ground truth

In [228]:
# needed info for later
# The elements that the consequents can not have,
# i.e. objects and scene and OCR because we only want to get private info.
list_semantic_objects = []
for elem_semantic in list_semantic:
    list_semantic_objects += elem_semantic
list_scene_objects = []
for elem_scene in list_scene:
    list_scene_objects += [elem_scene[0]]
not_in_consequents = list(set(list_semantic_objects)) + list(set(list_scene_objects))
list_private_objects = []
for elem_priv in list_GT_infos:
    list_private_objects += elem_priv
not_in_antecedents = list(set(list_private_objects))

In [230]:
not_in_antecedents = frozenset(not_in_antecedents) #list(frozenset([i]) for i in not_in_antecedents)
not_in_consequents = frozenset(not_in_consequents) #list(frozenset([i]) for i in not_in_consequents)

In [298]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

def getRules(dataset, min_support_score=0.6, min_lift_score=1.2, min_confidence_score=0.75):
    # Get the frequent item set.
    te = TransactionEncoder()
    te_ary = te.fit(dataset).transform(dataset)
    df = pd.DataFrame(te_ary, columns=te.columns_)
    frequent_itemsets = apriori(df, min_support=min_support_score, use_colnames=True)

    # Post filter the rules, for instance to use two metrics
    #association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)
    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=min_lift_score)
    rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
    rules = rules[ (rules['antecedent_len'] >= 0) &
           (rules['confidence'] > min_confidence_score) &
           (rules['lift'] > min_lift_score) ]
    return rules, frequent_itemsets

def getPredictionRules(rules, list_not_in_consequents, list_not_in_antecedents):
    #rules[rules['consequents'] == {'Eggs', 'Kidney Beans'}]
    # Certain elements can not be in consequents:
    # The dataframes use frozensets!!!
    #lambda f: 1 if  len(f.intersection(['a','b']))>0 else 0
    idx = rules['consequents'].apply(lambda f: False if len(f.intersection(list_not_in_consequents))>0 else True)
    filtered_rules = rules.loc[idx, :]
    #filtered_rules = rules.loc(rules['consequents'].apply(lambda f: 0 ))#if len(f.intersection(list_not_in_consequents))>0 else 1))# 0 if  len(f["consequents"].intersection(list_not_in_consequents))>0 else 1)
    #filtered_rules = rules.loc[~rules['consequents'].isin(list_not_in_consequents)]
    # Certain elements can not be in antecedents:
    return filtered_rules.loc[filtered_rules['antecedents'].apply(lambda f: False if len(f.intersection(list_not_in_antecedents))>0 else True)]

In [299]:
rules,frequent_itemsets = getRules(dataset_semantic_scene, 0.2, 0.5, 0.5)
filtered_rules = getPredictionRules(rules, not_in_consequents, not_in_antecedents)

In [301]:
display(filtered_rules)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
5,"(person, individual, someone, somebody, mortal, soul)",(a105_face_all),0.600962,0.403846,0.355769,0.592000,1.465905,0.113073,1.461161,1
15,(ceiling),(a109_person_body),0.322115,0.543269,0.201923,0.626866,1.153877,0.026928,1.224038,1
19,"(person, individual, someone, somebody, mortal, soul)",(a109_person_body),0.600962,0.543269,0.475962,0.792000,1.457841,0.149478,2.195821,1
23,(wall),(a109_person_body),0.918269,0.543269,0.500000,0.544503,1.002270,0.001133,1.002708,1
73,"(person, individual, someone, somebody, mortal, soul)","(a109_person_body, a105_face_all)",0.600962,0.403846,0.355769,0.592000,1.465905,0.113073,1.461161,1
88,"(floor, flooring, person, individual, someone, somebody, mortal, soul)",(a105_face_all),0.370192,0.403846,0.206731,0.558442,1.382808,0.057230,1.350113,2
100,"(person, individual, someone, somebody, mortal, soul, wall)",(a105_face_all),0.548077,0.403846,0.322115,0.587719,1.455305,0.100777,1.445990,2
112,"(ceiling, wall)",(a109_person_body),0.322115,0.543269,0.201923,0.626866,1.153877,0.026928,1.224038,2
118,"(floor, flooring, person, individual, someone, somebody, mortal, soul)",(a109_person_body),0.370192,0.543269,0.278846,0.753247,1.386507,0.077732,1.850962,2
130,"(signboard, sign, person, individual, someone, somebody, mortal, soul)",(a109_person_body),0.326923,0.543269,0.240385,0.735294,1.353462,0.062777,1.725427,2


In [302]:
pd.set_option('display.max_colwidth',-1)
display(filtered_rules[["antecedents", "consequents", "confidence", "lift"]])

c:\users\agathe~1\envs\privac~2\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,antecedents,consequents,confidence,lift
5,"(person, individual, someone, somebody, mortal, soul)",(a105_face_all),0.592000,1.465905
15,(ceiling),(a109_person_body),0.626866,1.153877
19,"(person, individual, someone, somebody, mortal, soul)",(a109_person_body),0.792000,1.457841
23,(wall),(a109_person_body),0.544503,1.002270
73,"(person, individual, someone, somebody, mortal, soul)","(a109_person_body, a105_face_all)",0.592000,1.465905
88,"(floor, flooring, person, individual, someone, somebody, mortal, soul)",(a105_face_all),0.558442,1.382808
100,"(person, individual, someone, somebody, mortal, soul, wall)",(a105_face_all),0.587719,1.455305
112,"(ceiling, wall)",(a109_person_body),0.626866,1.153877
118,"(floor, flooring, person, individual, someone, somebody, mortal, soul)",(a109_person_body),0.753247,1.386507
130,"(signboard, sign, person, individual, someone, somebody, mortal, soul)",(a109_person_body),0.735294,1.353462


In [303]:
print(len(rules), len(filtered_rules))

208 18
